# --- Day 15: Beacon Exclusion Zone ---

You feel the ground rumble again as the distress signal leads you to a large network of subterranean tunnels. You don't have time to search them all, but you don't need to: your pack contains a set of deployable sensors that you imagine were originally built to locate lost Elves.

The sensors aren't very powerful, but that's okay; your handheld device indicates that you're close enough to the source of the distress signal to use them. You pull the emergency sensor system out of your pack, hit the big button on top, and the sensors zoom off down the tunnels.

Once a sensor finds a spot it thinks will give it a good reading, it attaches itself to a hard surface and begins monitoring for the nearest signal source beacon. Sensors and beacons always exist at integer coordinates. Each sensor knows its own position and can determine the position of a beacon precisely; however, sensors can only lock on to the one beacon closest to the sensor as measured by the Manhattan distance. (There is never a tie where two beacons are the same distance to a sensor.)

It doesn't take long for the sensors to report back their positions and closest beacons (your puzzle input). For example:
```
Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3
```
So, consider the sensor at 2,18; the closest beacon to it is at -2,15. For the sensor at 9,16, the closest beacon to it is at 10,16.

Drawing sensors as S and beacons as B, the above arrangement of sensors and beacons looks like this:
```
               1    1    2    2
     0    5    0    5    0    5
 0 ....S.......................
 1 ......................S.....
 2 ...............S............
 3 ................SB..........
 4 ............................
 5 ............................
 6 ............................
 7 ..........S.......S.........
 8 ............................
 9 ............................
10 ....B.......................
11 ..S.........................
12 ............................
13 ............................
14 ..............S.......S.....
15 B...........................
16 ...........SB...............
17 ................S..........B
18 ....S.......................
19 ............................
20 ............S......S........
21 ............................
22 .......................B....
```
This isn't necessarily a comprehensive map of all beacons in the area, though. Because each sensor only identifies its closest beacon, if a sensor detects a beacon, you know there are no other beacons that close or closer to that sensor. There could still be beacons that just happen to not be the closest beacon to any sensor. Consider the sensor at 8,7:
```
               1    1    2    2
     0    5    0    5    0    5
-2 ..........#.................
-1 .........###................
 0 ....S...#####...............
 1 .......#######........S.....
 2 ......#########S............
 3 .....###########SB..........
 4 ....#############...........
 5 ...###############..........
 6 ..#################.........
 7 .#########S#######S#........
 8 ..#################.........
 9 ...###############..........
10 ....B############...........
11 ..S..###########............
12 ......#########.............
13 .......#######..............
14 ........#####.S.......S.....
15 B........###................
16 ..........#SB...............
17 ................S..........B
18 ....S.......................
19 ............................
20 ............S......S........
21 ............................
22 .......................B....
```
This sensor's closest beacon is at 2,10, and so you know there are no beacons that close or closer (in any positions marked #).

None of the detected beacons seem to be producing the distress signal, so you'll need to work out where the distress beacon is by working out where it isn't. For now, keep things simple by counting the positions where a beacon cannot possibly be along just a single row.

So, suppose you have an arrangement of beacons and sensors like in the example above and, just in the row where y=10, you'd like to count the number of positions a beacon cannot possibly exist. The coverage from all sensors near that row looks like this:
```
                 1    1    2    2
       0    5    0    5    0    5
 9 ...#########################...
10 ..####B######################..
11 .###S#############.###########.
```
In this example, in the row where y=10, there are 26 positions where a beacon cannot be present.

Consult the report from the sensors you just deployed. In the row where y=2000000, how many positions cannot contain a beacon?

In [ ]:
import logging
import sys

sys.path.append("..")
from aoc import get_logger, get_data, timer, test

logger = get_logger("day15", logging.INFO)
data = get_data("day15")

logger.info("initialization complete!")

## Solution Part One

In [ ]:
import re
from tqdm import tqdm


class Map(object):
    def __init__(self, input_: list):
        self.input = input_
        self.map = dict()
        self.sensors = dict()
        self.frame = [sys.maxsize, -sys.maxsize, sys.maxsize, -sys.maxsize]  # init x-min, x-max, y-min, y-max

    def scan(self):
        for line in self.input:
            sx, sy, bx, by = map(int, re.findall(r"[+-]?\d+", line))
            sensor = (sx, sy)
            beacon = (bx, by)
            self.map[sensor] = "S"
            self.map[beacon] = "B"
            sensor_range = abs(sx - bx) + abs(sy - by)
            self.frame[0] = min(sx - sensor_range, bx, self.frame[0])
            self.frame[1] = max(sx + sensor_range, bx, self.frame[1])
            self.frame[2] = min(sx - sensor_range, bx, self.frame[2])
            self.frame[3] = max(sx + sensor_range, bx, self.frame[3])
            self.sensors[sensor] = sensor_range

    def filter_sensors(self):
        self.sensors = {
            (sx, sy): sensor_range
            for (sx, sy), sensor_range in self.sensors.items()
            if sx - sensor_range <= self.frame[1]
            and sx + sensor_range >= self.frame[0]
            and sy - sensor_range <= self.frame[3]
            and sy + sensor_range >= self.frame[2]
        }

    def mark_exclusion_zones(self):
        exclusion_zone = set()
        for (sx, sy), sensor_range in tqdm(self.sensors.items()):
            for x in range(max(sx - sensor_range, self.frame[0]), min(sx + sensor_range, self.frame[1]) + 1):
                for y in range(max(sy - sensor_range, self.frame[2]), min(sy + sensor_range, self.frame[3]) + 1):
                    if abs(sx - x) + abs(sy - y) <= sensor_range:
                        if x < self.frame[0] or x > self.frame[1] or y < self.frame[2] or y > self.frame[3]:
                            continue
                        exclusion_zone.add((x, y))
        for (x, y) in exclusion_zone:
            if (x, y) not in self.map:
                self.map[(x, y)] = "#"

    def count(self, item: str):
        return sum([cell == item for row in self.get_frame() for cell in row])

    def get_frame(self) -> list:
        matrix = []
        for y in range(self.frame[2], self.frame[3] + 1):
            row = []
            for x in range(self.frame[0], self.frame[1] + 1):
                row.append(self.map.get((x, y), "."))
                # print(self.map.get((x, y), "."), end="")
            matrix.append(row)
        return matrix

    def __repr__(self):
        return "\n".join(["".join(row) for row in self.get_frame()])


@timer
def part1(input_) -> int:
    input_ = input_.split("\n")
    map = Map(input_)
    map.scan()

    if len(input_) < 20:  # example input
        row_number = 10
        map.frame[2] = row_number
        map.frame[3] = row_number
        map.filter_sensors()
        map.mark_exclusion_zones()
        print(map)
    else:
        row_number = 2000000
        map.frame[2] = row_number
        map.frame[3] = row_number
        map.filter_sensors()
        map.mark_exclusion_zones()
    return map.count("#")


test(func=part1, input_=data.example_input, expected=data.test_answer1)

print("solution part 1:", part1(data.input))

# --- Part Two ---

Your handheld device indicates that the distress signal is coming from a beacon nearby. The distress beacon is not detected by any sensor, but the distress beacon must have x and y coordinates each no lower than 0 and no larger than 4000000.

To isolate the distress beacon's signal, you need to determine its tuning frequency, which can be found by multiplying its x coordinate by 4000000 and then adding its y coordinate.

In the example above, the search space is smaller: instead, the x and y coordinates can each be at most 20. With this reduced search area, there is only a single position that could have a beacon: x=14, y=11. The tuning frequency for this distress beacon is 56000011.

Find the only possible position for the distress beacon. What is its tuning frequency?

## Solution Part Two
The solution to part one was way too slow for part two, so instead of checking all the cells within the frame, we're only going to check the cells that are just outside the range of the sensors.

In [ ]:
def is_in_range(x, y, sx, sy, sensor_range):
    """returns True if (x, y) is in range of sensor (sx, sy)"""
    return abs(sx - x) + abs(sy - y) <= sensor_range


class Map2(Map):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_sensor_outline(self, sx, sy, sensor_range) -> set:
        """returns the outer border of the sensor's range"""
        outline = set()
        for y_outline in range(max(sy - sensor_range, self.frame[2]), min(sy + sensor_range, self.frame[3]) + 1):
            x_outline_left = sx - (sensor_range - abs(y_outline - sy)) - 1
            outline.add((x_outline_left, y_outline)) if self.frame[0] <= x_outline_left <= self.frame[1] else None
            x_outline_right = sx + (sensor_range - abs(y_outline - sy)) + 1
            outline.add((x_outline_right, y_outline)) if self.frame[0] <= x_outline_right <= self.frame[1] else None
        top_y = sy - sensor_range - 1
        outline.add((sx, top_y)) if self.frame[2] <= top_y <= self.frame[3] else None
        bottom_y = sy + sensor_range + 1
        outline.add((sx, bottom_y)) if self.frame[2] <= bottom_y <= self.frame[3] else None
        return outline

    def sparse_search_out_of_range(self):
        """Check if there is a point in the outer border of the sensors that is not in range of any sensor"""
        for (sx, sy), sensor_range in tqdm(self.sensors.items()):
            outline = self.get_sensor_outline(sx, sy, sensor_range)

            last_sensor = (sx, sy)
            for x, y in outline:
                # next sensor is likely to be the same as the last one
                if is_in_range(x, y, *last_sensor, self.sensors[last_sensor]):
                    continue

                for sensor in self.sensors:
                    if sensor == (sx, sy):
                        continue
                    if is_in_range(x, y, *sensor, self.sensors[sensor]):
                        last_sensor = sensor
                        break
                else:
                    return x, y
        raise Exception("no solution found")


@timer
def part2(input_) -> int:
    input_ = input_.split("\n")
    map2 = Map2(input_)
    map2.scan()

    if len(input_) < 20:  # example input
        map2.frame = [0, 20, 0, 20]
    else:
        map2.frame = [0, 4000000, 0, 4000000]
    map2.filter_sensors()
    x, y = map2.sparse_search_out_of_range()
    print("x:", x, "y:", y)
    return x * 4000000 + y


test(func=part1, input_=data.example_input, expected=data.test_answer1)

print("solution part 2:", part2(data.input))